In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
from sklearn.model_selection import KFold

kkbox-music-recommendation-challenge
music-recommendation-kkbox



In [2]:
df_test = pd.read_csv('../input/music-recommendation-kkbox/test.csv',dtype={'msno' : 'category',
												'source_system_tab' : 'category',
                                                  'source_screen_name' : 'category',
                                                  'source_type' : 'category',
                                                'song_id' : 'category'})
df_test.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [3]:
df_train = pd.read_csv('../input/music-recommendation-kkbox/train.csv',dtype={'msno' : 'category',
													'source_system_tab' : 'category',
                                                  'source_screen_name' : 'category',
                                                  'source_type' : 'category',
                                                  'target' : np.uint8,
                                                  'song_id' : 'category'})
df_train.head()
                                                                            

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [4]:
df_members = pd.read_csv('../input/music-recommendation-kkbox/members.csv',dtype={'city' : 'category',
                                                      'bd' : 'category',
                                                      'gender' : 'category',
                                                      'registered_via' : 'category'})
df_members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [5]:
df_members['registration_year'] = df_members['registration_init_time'].apply(lambda x : int(str(x)[0:4]))
df_members['registration_year'] = pd.to_numeric(df_members['registration_year'],downcast='unsigned')

df_members['registration_month'] = df_members['registration_init_time'].apply(lambda x : int(str(x)[4:6]))
df_members['registration_month'] = pd.to_numeric(df_members['registration_month'],downcast='unsigned')

df_members['registration_date'] = df_members['registration_init_time'].apply(lambda x : int(str(x)[6:8]))
df_members['registration_date'] = pd.to_numeric(df_members['registration_date'],downcast='unsigned')

df_members.drop('registration_init_time',axis=1,inplace=True)

df_members['expiration_year'] = df_members['expiration_date'].apply(lambda x : int(str(x)[0:4]))
df_members['expiration_year'] = pd.to_numeric(df_members['expiration_year'],downcast='unsigned')

df_members['expiration_month'] = df_members['expiration_date'].apply(lambda x :int(str(x)[4:6]))
df_members['expiration_month'] = pd.to_numeric(df_members['expiration_month'],downcast='unsigned')

df_members['expiration_date'] = df_members['expiration_date'].apply(lambda x : int(str(x)[6:8]))
df_members['expiration_date'] = pd.to_numeric(df_members['expiration_date'],downcast='unsigned')

df_test = pd.merge(left = df_test,right = df_members,how='left',on='msno')
df_test.msno = df_test.msno.astype('category')
df_train = pd.merge(left = df_train,right = df_members,how='left',on='msno')
df_train.msno = df_train.msno.astype('category')

del df_members

print("Step Complete")

Step Complete


In [6]:
df_songs = pd.read_csv('../input/music-recommendation-kkbox/songs.csv',dtype={'genre_ids': 'category',
                                                  'language' : 'category',
                                                  'artist_name' : 'category',
                                                  'composer' : 'category',
                                                  'lyricist' : 'category',
                                                  'song_id' : 'category'})
df_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [7]:
df_test = pd.merge(left = df_test,right = df_songs,how = 'left',on='song_id')
df_test.song_length.fillna(200000,inplace=True)
df_test.song_length = df_test.song_length.astype(np.uint32)
df_test.song_id = df_test.song_id.astype('category')

df_train = pd.merge(left = df_train,right = df_songs,how = 'left',on='song_id')
df_train.song_length.fillna(200000,inplace=True)
df_train.song_length = df_train.song_length.astype(np.uint32)
df_train.song_id = df_train.song_id.astype('category')

del df_songs

print("Step Complete")

Step Complete


In [8]:
import lightgbm as lgb
kf = KFold(n_splits=3)
predictions = np.zeros(shape=[len(df_test)])
print("Step Complete")

Step Complete


In [9]:
for train_indices,val_indices in kf.split(df_train) : 
    train_data = lgb.Dataset(df_train.drop(['target'],axis=1).loc[train_indices,:],label=df_train.loc[train_indices,'target'])
    val_data = lgb.Dataset(df_train.drop(['target'],axis=1).loc[val_indices,:],label=df_train.loc[val_indices,'target'])
    
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.1 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 128,
        'max_depth': 10,
        'num_rounds': 200,
        } 
    
    bst = lgb.train(params, train_data, 100, valid_sets=[val_data])
    predictions+=bst.predict(df_test.drop(['id'],axis=1))
    del bst

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.182329 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's binary_logloss: 0.717608
[2]	valid_0's binary_logloss: 0.707899
[3]	valid_0's binary_logloss: 0.699837
[4]	valid_0's binary_logloss: 0.69273
[5]	valid_0's binary_logloss: 0.687126
[6]	valid_0's binary_logloss: 0.681639
[7]	valid_0's binary_logloss: 0.676889
[8]	valid_0's binary_logloss: 0.672857
[9]	valid_0's binary_logloss: 0.669536
[10]	valid_0's binary_logloss: 0.666064
[11]	valid_0's binary_logloss: 0.663193
[12]	valid_0's binary_logloss: 0.660836
[13]	valid_0's binary_logloss: 0.657977
[14]	valid_0's binary_logloss: 0.655978
[15]	valid_0's binary_logloss: 0.654374
[16]	valid_0's binary_logloss: 0.652303
[17]	valid_0's binary_logloss: 0.650553
[18]	valid_0's binary_logloss: 0.649385
[19]	valid_0's binary_logloss: 0.648003
[20]	valid_0's binary_logloss: 0.646552
[21]	valid_0's binary_logloss: 0.645464
[22]	valid_0's binary_logloss: 0.644357
[23]	valid_0's binary_logloss: 0.643659
[24]	valid_0's binary_logloss: 0.64264
[25]	valid_0's binary_logloss: 0.641969
[26]	valid_

In [10]:
predictions = predictions/3

submission = pd.read_csv('../input/music-recommendation-kkbox/sample_submission.csv')
submission.target=predictions
submission.to_csv('submission.csv',index=False)
print("Step Complete")

Step Complete
